In [1]:
# import required libraries
import pandas as pd
import string
from nltk.corpus import stopwords

In [2]:
#Get the spam data collection 
dt_spamCollection = pd.read_csv('C:\Simplilearn\Data-Science-Python\Projects\Project-08\Lesson 9 -1\SpamCollection\SpamCollection',header =None,sep ='\t',
                               names=['response','message'])

In [3]:
# view the 5 first records of data
dt_spamCollection.head()

,response,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
# view the informations of the data

dt_spamCollection.describe()
#dt_spamCollection['message']

,response,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [5]:
#view response 
dt_spamCollection.groupby('response').describe()

message                                                            \
           count unique                                                top   
response                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
response       
ham        30  
spam        4

In [6]:
#Verify length of the messages and also add it as a new column 
dt_spamCollection['length'] = dt_spamCollection['message'].apply(len)

In [7]:
dt_spamCollection.head()

,response,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [8]:
#define a function to get rid of stopwords present in the messages
def removestopwords(messg):
    no_ponctuation = [char for char in messg if char not in string.punctuation]
    no_ponctuation = ''.join(no_ponctuation)
    return [word for word in no_ponctuation.split() if word.lower() not in stopwords.words('english')] 

In [9]:
#test the function
#print(removestopwords('i say hello Ebrahim '))
# apply the function to the data
dt_spamCollection['message'].head(5).apply(removestopwords)


0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [10]:
#start text processing with vectorizer 
from sklearn.feature_extraction.text import CountVectorizer

In [26]:
#use bag of words by applying the function and fit the data into it
bag_Of_Words_transformer = CountVectorizer(analyzer=removestopwords).fit(dt_spamCollection['message'])

In [27]:
#print length of bag of words stored in the vocabulary_ attribute
print (len(bag_Of_Words_transformer.vocabulary_))

11425


In [30]:
message_bagOfWords =bag_Of_Words_transformer.transform(dt_spamCollection['message']) 


In [31]:
#apply tfidf transformer and fit the bag of words into it (transformed version)
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(message_bagOfWords)

In [32]:
#print shape of the tfidf 
message_Tfid = tfidf_transformer.transform(message_bagOfWords)
print(message_Tfid.shape)

(5572, 11425)


In [33]:
#choose naive Bayes model to detect the spam and fit the tfidf data into it
from sklearn.naive_bayes import MultinomialNB
spam_model_detection = MultinomialNB().fit(message_Tfid,dt_spamCollection['response'])

In [37]:
#check model for the predicted and expected value say for message#2 and message#5
message = dt_spamCollection['message'][2]

bag_of_word_for_message = bag_Of_Words_transformer.transform([message])
tfid = tfidf_transformer.transform(bag_of_word_for_message)
print('Predict second message :',spam_model_detection.predict(tfid)[0])
print('Expected :',dt_spamCollection['response'][2])


Predict second message : spam
Expected : spam
